<div style="text-align: right" align="right"><i>Peter Norvig, December 2025</i></div>

# Advent of Code 2025

I  enjoy doing the [**Advent of Code**](https://adventofcode.com/) (AoC) programming puzzles, so here we go for 2025!  This year I will be doing something different: I will solve each problem, and then [**in another notebook**](Advent-2025-AI.ipynb) I will ask an AI Large Language Model to solve the same problem, and I'll comment on it. So far the LLMs are doing great, but IMHO I prefer my code.

# Day 0

I'm glad that [@GaryGrady](https://mastodon.social/@garygrady) is providing cartoons:

<a href="https://x.com/garyjgrady"><img src="https://pbs.twimg.com/media/Gdp709FW8AAq2_m?format=jpg&name=medium" width=500 alt="Gary Grady cartoon"></a>

I start by loading up my [**AdventUtils.ipynb**](AdventUtils.ipynb) notebook (same as last time except for the `current_year`). On each day I will first parse the input (with the help of my `parse` utility function), then solve Part 1 and Part 2 (recording the correct answer with my `answer` function).

In [1]:
%run AdventUtils.ipynb
current_year = 2025

# [Day 1](https://adventofcode.com/2025/day/1): Secret Entrance

On Day 1 we meet an elf and learn that our task is to finish decorating the North Pole by December 12th. There will be 24 challenges along the way. Today we need to unlock a safe. The safe has a dial with 100 numbers. Our input for today is a sequence of left and right rotations; for example "R20" means move the dial right by 20 numbers and "L13" means move it left by 13 numbers. I'll use my `parse` utility function to parse each line of the input as an integer, after replacing each 'L' with a minus sign and each 'R' with a plus sign:

In [2]:
def parse_rotation(line: str): return int(line.replace('L', '-').replace('R', '+'))

rotations = parse(day=1, parser=parse_rotation) 

────────────────────────────────────────────────────────────────────────────────────────────────────
Puzzle input ➜ 4780 strs:
────────────────────────────────────────────────────────────────────────────────────────────────────
L20
L13
L16
L16
L29
L7
L48
L48
...
────────────────────────────────────────────────────────────────────────────────────────────────────
Parsed representation ➜ 4780 ints:
────────────────────────────────────────────────────────────────────────────────────────────────────
-20
-13
-16
-16
-29
-7
-48
-48
...


Note that my `parse` function prints the first few input lines, then prints the parsed representation of these lines. That helps me debug and helps you the reader understand what is going on.

<img src="https://files.mastodon.social/media_attachments/files/115/646/343/679/448/846/original/428b312ca88f62c4.jpg" width=500 alt="Gary Grady cartoon">

### Part 1: How many times is the dial left pointing at 0 after any rotation in the sequence?

Initially the safe's arrow is pointing at 50, and then we apply the rotations in order. We are asked how many of the rotations leave the dial pointing at 0. The `itertools.accumulate` function yields running totals of its input sequence, so we just have to count (quantify) how many times the running total of the rotations is 0 mod 100. I'll write a function to do that and run it on the input:

In [3]:
def count_zeros(numbers, dial=100) -> int:
    """How many zeros (modulo `dial`) in the running partial sums of the numbers?"""
    return quantify(total % dial == 0 for total in accumulate(numbers, initial=50))

In [4]:
count_zeros(rotations)

1182

I submitted "1182" to AoC and saw it was correct, so I record the `answer` like this:

In [5]:
answer(puzzle=1.1, solution=1182, code=lambda: 
       count_zeros(rotations))

Puzzle  1.1:   .0005 seconds, answer 1182            ok

### Part 2: How many times does the dial point to 0 at any time?

For Part 2 we need to count both when a rotation ends up at 0 and when the arrow passes 0 at any time during a rotation. For example, if the arrow points to 95, then only a "R5" or a "L95" would register a 0 in Part 1, but now a rotation of "R10" would also count because it passes 0 (as would any rotation of "R5" or larger, or "L95" or larger). 

I'll start with a simple but slow approach: treat a rotation of, say, -20 as 20 rotations of -1, and then use the same `count_zeros` function from part 1. (Note that `sign(r)` returns +1 for any positive input, and -1 for any negative input.)

In [6]:
answer(1.2, 6907, lambda:
       count_zeros(sign(r) for r in rotations for _ in range(abs(r))))

Puzzle  1.2:   .1389 seconds, answer 6907            ok

That's a long run time for a Day 1 problem, so here's a faster method: I break each rotation down into a number of full circles and some remainder, then add the full circles to the count of zeros, and add one more if the remainder is at least as much as the distance to zero (in the appropriate direction): 

In [7]:
def zero_clicks(rotations, position=50, dial=100) -> int:
    """How many times does any click cause the dial to point at 0?
    Count 1 if the rotation crosses the distance to 0,
    and for large rotations, count abs(r) // 100 more."""
    zeros = 0
    for r in rotations:
        full_circles, remainder = divmod(abs(r), dial)
        distance_to_0 = (dial - position if (r > 0 or position == 0) else position)
        zeros += full_circles + (1 if remainder >= distance_to_0 else 0)
        position = (position + r) % dial
    return zeros

In [8]:
answer(1.2, 6907, lambda:
       zero_clicks(rotations))

Puzzle  1.2:   .0009 seconds, answer 6907            ok

That's much faster, but the code is trickier, and indeed I initially had a **bug** in the `distance_to_0` computation: when the current position is 0 the distance should be 100: it takes a full rotation to get back to 0. My code initially claimed the distance was 0; adding `or position == 0` fixed that.

# [Day 2](https://adventofcode.com/2025/day/2): Gift Shop

Today we're in the North Pole gift shop, and are asked to help the elves identify invalid product IDs on the items there. We're giving a list of ranges of product IDs of items in stock. Each range is a pair of integers separated by a dash, and the ranges are separated by commas:

In [9]:
id_ranges = parse(day=2, parser=positive_ints, sections=lambda text: text.split(','))

────────────────────────────────────────────────────────────────────────────────────────────────────
Puzzle input ➜ 1 str:
────────────────────────────────────────────────────────────────────────────────────────────────────
990244-1009337,5518069-5608946,34273134-34397466,3636295061-3636388848,8613701-8663602,573252-68 ...
────────────────────────────────────────────────────────────────────────────────────────────────────
Parsed representation ➜ 35 tuples:
────────────────────────────────────────────────────────────────────────────────────────────────────
(990244, 1009337)
(5518069, 5608946)
(34273134, 34397466)
(3636295061, 3636388848)
(8613701, 8663602)
(573252, 688417)
(472288, 533253)
(960590, 988421)
...


<img src="https://files.mastodon.social/media_attachments/files/115/652/152/368/251/243/original/56e4ed8e5f24db96.jpg" width=500 alt="GaryJGrady cartoon">

### Part 1: What is the sum of the invalid IDs?

An invalid ID is defined as one that consists of a digit sequence repeated twice. So 55, 6464 and 123123 are invalid. We're asked for the sum of the invalid IDs across all the ID ranges.

We could look at every number in each range and check if the first half of the number (as a string) is the same as the second half. How many checks would that be?

In [10]:
sum((hi - lo + 1) for lo, hi in id_ranges)

1990936

Only 2 million! So it would indeed be feasible to check every one. But I have a suspicion that Part 2 would make it infeasible, so I'll invest in a more efficient approach. For each ID range, instead of enumerating every number in the range and checking each one for validity, I will instead enumerate over the *first half* of the possible digit strings, and automatically generate invalid IDs by appending a copy of the first half to itself. By *first half* I don't mean divide by 2; I mean the first half of the digit string: the first half of "123456" is "123".

Suppose the range is 123456-223000.  I enumerate from 123 to 223, and for each number generate an invalid ID:
[123123, 124124, 125125, ... 223223]. I then yield the IDs that are within the range (in this case all but the first and the last are in the range 123456-223000). Altogether I only have to consider 101 IDs rather than 100,001. (The algorithm scales with the square root of the size of the range, not with the size of the range itself.)

In [11]:
def invalids_in_range(lo: int, hi: int) -> Iterable[int]:
    """Yield all the invalid IDs between lo and hi inclusive.
    An ID is invalid if it consists of a digit sequence repeated twice."""
    first_half = str(lo)[:max(1, len(str(lo)) // 2)]
    for i in count_from(int(first_half)):
        id = int(str(i) * 2)
        if lo <= id <= hi:
            yield id
        elif id > hi:
            return

def invalids(id_ranges) -> List[int]:
    """Invalid IDs, according to any one of the list of invalid ID ranges."""
    return append(invalids_in_range(lo, hi)
                  for (lo, hi) in id_ranges)

assert invalids([(11, 22)]) == [11, 22]

In [12]:
answer(2.1, 23560874270, lambda:
       sum(invalids(id_ranges)))

Puzzle  2.1:   .0031 seconds, answer 23560874270     ok

### Part 2: What is the sum of the invalid IDs, under the new rules?

In Part 2 we discover that an ID should be considered invalid if it consists of two *or more* repeats of a sequence of digits. So 111 (1 repeated three times), 12121212 (12 repeated four times), and 222222 (2 repeated six times) are all invalid. I'll rewrite `invalids_in_range` to take an optional argument saying how many repeats we're looking for, and introduce  `all_invalids` to try all possible repeat lengths:

In [13]:
def invalids_in_range(lo: int, hi: int, repeat=2) -> Iterable[int]:
    """Yield all the invalid IDs between lo and hi inclusive
    that are formed from exactly `repeat` repeated digit sequences."""
    first_half = int(str(lo)[:len(str(lo)) // repeat] or 1)
    for i in count_from(int(first_half)):
        id = int(str(i) * repeat)
        if lo <= id <= hi:
            yield id
        elif id > hi:
            return

def all_invalids(id_ranges) -> Set[int]:
    """All Invalid IDs, according to the list of ranges, with any number of repeats."""
    return union(invalids_in_range(lo, hi, repeat)
                 for (lo, hi) in id_ranges
                 for repeat in range(2, len(str(hi)) + 1))

Since this is a bit tricky, I include some unit test assertions:

In [14]:
assert list(invalids_in_range(2121212118, 2121212124, 5)) == [2121212121]
assert all_invalids([(11, 22), (95, 115)]) == {11, 22, 99, 111}

Now I'll verify that the answer for Part 1 still works, and go ahead and compute the answer for Part 2:

In [15]:
answer(2.1, 23560874270, lambda:
       sum(invalids(id_ranges)))

Puzzle  2.1:   .0030 seconds, answer 23560874270     ok

In [16]:
answer(2.2, 44143124633, lambda:
       sum(all_invalids(id_ranges)))

Puzzle  2.2:   .0041 seconds, answer 44143124633     ok

I initially had another **bug** here: I counted "222222" three times: once as 2 repeats of "222", once as 3 repeats of "22", and once as 6 repeats of "2". I changed the output of `all_invalids` to be a `set` to fix that.

# [Day 3](https://adventofcode.com/2025/day/3): Lobby

Entering the lobby, we find that the elevators are offline. We might be able to fix the problem by turning on some batteries. There are multiple battery banks, each bank consisting of a sequence of batteries, each labeled with its *joltage*, a digit from 1 to 9. 

In [17]:
banks = parse(day=3)

────────────────────────────────────────────────────────────────────────────────────────────────────
Puzzle input ➜ 200 strs:
────────────────────────────────────────────────────────────────────────────────────────────────────
5353323523322232362334333433323333353233331313222372133133353643423323233323333534414523333432223242
6344544745655555456556556566665564538465555575558846455665837545764555554465564547547565544657585435
2246273372253242254243532252231242225522622633532222322234255122531222423531343223123232234213323424
6545643634344444495734739454433454439454355654483544243344534445434437426443854344454534654439534424
2356636643143433535443636338231745346538433576334436353176353333433532345344334224435234343644332536
3221311221443323323322222214632342232233222322333436263122265162212321261323142262212332322125216222
3336332333336335335324359336493238433441666379243536334165623214253384333323893933867663434332383763
323532125233243133222323243622253243222622322221323343285353532231

### Part 1: What is the maximum possible total output joltage?

We can turn on exactly two batteries in each bank, resulting in a two digit number which is the *joltage* of the bank. For example, given the bank "8647" we could choose to turn on the "8" and "7" to produce a joltage of 87; that's the maximum.  The function `joltage` chooses the biggest first digit, and then the biggest second digit that follows the first digit. Note that the possible choices for the first digit exclude the last digit, because if we chose that, then there would be no choices left for the second digit. (I chose to do the string-to-int conversion in `total_joltage`; it would also be fine to have `joltage` return an int.)

In [18]:
def joltage(bank: str) -> str:
    """The maximum possible joltage by turning on 2 batteries in the bank.
    Pick the biggest first digit, then the biggest digit that follows."""
    first  = max(bank[:-1]) # The first digit can't be the last character
    second = max(bank[bank.index(first) + 1:]) # The second digit must come after the first
    return first + second

def total_joltage(banks: List[str]) -> int:
    """The maximum possible joltage from all the banks."""
    return sum(int(joltage(bank)) for bank in banks)

In [19]:
assert joltage("8647") == "87"
assert joltage("1119") == "19"

In [20]:
answer(3.1, 17085, lambda:
       total_joltage(banks))

Puzzle  3.1:   .0006 seconds, answer 17085           ok

### Part 2: What is the new maximum possible total output joltage?

In Part 2 the elf hits the "*joltage limit safety override*" button, and we can now turn on 12 batteries per bank, resulting in a 12-digit joltage. What is the new maximum possible total joltage?

I will make a change to the two functions, passing in the number of digits to be chosen, *n* (with default 2 for backwards compatibility). The function `joltage` recurses when there is more than one digit remaining, choosing the first digit from the bank up to the last *n* - 1 characters, then recursively finding the biggest joltage from the rest. 

In [21]:
def joltage(bank: str, n=2) -> str:
    """The maximum possible joltage by turning on `n` batteries in the bank.
    Pick the first digit, then the maximum joltage from the rest of the bank."""
    if n == 1:
        return max(bank)
    else:
        first = max(bank[:-(n - 1)]) # The first digit can't be the last n-1 characters
        rest  = bank[bank.index(first) + 1:]
        return first + joltage(rest, n - 1)

def total_joltage(banks: List[str], n=2) -> int:
    """The maximum possible joltage from all the banks,
    when `n` batteries are turned on per bank."""
    return sum(int(joltage(bank, n)) for bank in banks)

In [22]:
assert joltage("811111111111119", 2)  == '89'
assert joltage("818181911112111", 5)  == '92111'
assert joltage("818181911112111", 12) == '888911112111'

I'll make sure Part 1 still works, and then solve Part 2:

In [23]:
answer(3.1, 17085, lambda:
       total_joltage(banks))

Puzzle  3.1:   .0006 seconds, answer 17085           ok

In [24]:
answer(3.2, 169408143086082, lambda:
       total_joltage(banks, 12))

Puzzle  3.2:   .0020 seconds, answer 169408143086082 ok

# [Day 4](https://adventofcode.com/2025/day/4): Printing Department

The floor of the printing department is divided into squares, some of which contain a roll of paper. The day's input is a map of the floor, with `@` representing a roll of paper. I can handle that with the `Grid` class from my AdventUtils:

In [25]:
paper_grid = Grid(parse(day=4), directions=directions8)

────────────────────────────────────────────────────────────────────────────────────────────────────
Puzzle input ➜ 140 strs:
────────────────────────────────────────────────────────────────────────────────────────────────────
.@@@@@...@.@@@@@@@@@@.@@@@@@@.@.@.@@@@@@@@@@@@@..@.@@@.@@@@@@..@.@..@.@@...@.@@@@..@@@@....@@@.@ ...
.@@@@@.@....@.....@@@.@@.@@.@@@.@@@.@.@.@.@@@@@.@@.@@@@@.@@@@@@@@@@@..@@.@.@@.@@@.@@@@@@@@@@@..@ ...
@.@@@@.@@@@.@@@@..@@.@@@@@@@@.@@@@.@@@@.@@..@.@...@.@.@.@.@@..@@@@@.@.@.@@@@.@@@@@@@@@.@@@@..@@. ...
.@.....@.@@@..@.@@@.@..@@@@@..@@@.@@..@...@.@@@@.@@@.@.@@@@@@.@.@@@@@@@.@.@@@.@@@@@@...@@.@@..@. ...
@@@@@.@@@.@@@@@@@..@@.@.@@@..@@..@@@.@@....@.@..@@@@@@@@.@.@@..@@...@@.@@@...@.@.@@@..@.@.@@@@@@ ...
@.@@@@@@..@@@@...@..@@@@@@.@@@..@.....@@.@.@@...@@@.@@.@.@@@....@@.@.@.@@@@.@@@@@.@@@.@@...@@.@@ ...
.@@@.@.@@@..@@.@.@@@@@.@.@..@@....@..@.@.@@@@.@..@@.@..@@@@@.@@@@@@@.@.@@@.@.@@@.@@@@.@@@@@@@@.@ ...
@@@@@@@.@@...@@@....@.@@@@.@@@@@@@@@.@@@.@@.@@..@...@@@@@.@@@..@.@

<img src="https://files.mastodon.social/media_attachments/files/115/663/375/845/834/867/original/4d3180e12fa726fe.jpg" width=500 alt="Gary Grady cartoon">


### Part 1: How many rolls of paper can be accessed by a forklift?

A roll is **accessible** by forklift if there are fewer than four rolls of paper in the eight adjacent positions (that's why I specified `directions8` in defining `paper_grid`). Counting the number of accessible rolls is easy, but I decided to make `accessible rolls` return a list of positions rather than a count, in anticipation of Part 2.

In [26]:
def accessible_rolls(grid: Grid) -> List[Point]:
    """The positions of all the accessible rolls of paper on the grid."""
    return [p for p in grid if is_accessible(p, grid)]

def is_accessible(p: Point, grid: Grid) -> bool:
    """A roll of paper is accessible if there are fewer than 
    four rolls of paper in the eight adjacent positions."""
    return grid[p] == '@' and grid.neighbor_contents(p).count('@') < 4

Here's the answer:

In [27]:
answer(4.1, 1569, lambda:
       len(accessible_rolls(paper_grid)))

Puzzle  4.1:   .0541 seconds, answer 1569            ok

### Part 2: How many rolls of paper can be removed?

If the elves can access a paper roll, they can remove it by forklift. That may in turn make other rolls accessible, and hence removable. How many rolls in total can be removed?

It looks like I was right to make `accessible_rolls` return a list of points rather than a count! I can answer the question by repeatedly finding the accessible rolls, removing them (on a copy of the grid so I don't mess up the original grid), and repeating until there are no more accessible rolls.

In [28]:
def removable_rolls(grid: Grid) -> Iterable[Point]:
    """The positions of paper rolls that can be removed, in any nuber of iterations."""
    grid2 = grid.copy() # To avoid mutating the input grid
    points = accessible_rolls(grid2)
    while points:
        yield from points
        grid2.update({p: '.' for p in points})
        points = accessible_rolls(grid2)

In [29]:
answer(4.2, 9280, lambda:
       quantify(removable_rolls(paper_grid)))     

Puzzle  4.2:  1.2345 seconds, answer 9280            ok

That's the right answer, but the run time is slow. The main issue is that `accessible_rolls` has to look at the whole grid on every iteration. That's wasteful: If the previous iteration only removed one roll, all we really need to look at on the next iteration is the neighbors of the removed roll. So I'll keep a queue of possibly removable points and repeatedly pop a point off the queue, and if it is an accessible roll, remove it and put all its neighbors on the queue. When the queue is empty, no more rolls can be removed.

In [30]:
def removable_rolls(grid: Grid) -> Iterable[Point]:
    """The positions of paper rolls that can be removed, in any nuber of iterations."""
    grid2 = grid.copy() # To avoid mutating the original input grid
    Q = list(grid)      # A queue of possibly removable positions in the grid
    while Q:
        p = Q.pop()
        if is_accessible(p, grid2):
            yield p
            grid2[p] = '.'
            Q.extend(grid2.neighbors(p))

In [31]:
answer(4.2, 9280, lambda:
       quantify(removable_rolls(paper_grid)))

Puzzle  4.2:   .1421 seconds, answer 9280            ok

# [Day 5](https://adventofcode.com/2025/day/5): Cafeteria

Today we're in the cafeteria, and the elves need to figure out which of their ingredients are fresh or spoiled. The input file has two parts (paragraphs), the first consisting of ranges of fresh ingredient IDs, like "3-5" and the second consisting of the available ingredient IDs, like "8". I can parse the data like this:

In [32]:
def parse_ingredients(text: str) -> tuple:
    """Parse either ranges like "3-5", or integers like "8"."""
    return mapt(positive_ints, lines(text)) if '-' in text else ints(text)
                                                   
fresh_ranges, available_ingredient_ids = parse(day=5, parser=parse_ingredients, sections=paragraphs)

────────────────────────────────────────────────────────────────────────────────────────────────────
Puzzle input ➜ 1184 strs:
────────────────────────────────────────────────────────────────────────────────────────────────────
292632986393425-296797126337251
428261559408337-431275643240865
197704206528056-198822557342819
36791726875734-37049023408764
134880223152389-139959748438608
31870818340663-32138457068292
443613579514078-447616030778273
284142407577672-288117756872436
...
────────────────────────────────────────────────────────────────────────────────────────────────────
Parsed representation ➜ 2 tuples:
────────────────────────────────────────────────────────────────────────────────────────────────────
((292632986393425, 296797126337251), (428261559408337, 431275643240865), (197704206528056, 19882 ...
(92087202605588, 314304587960778, 19184152482180, 331502336245673, 104093299503920, 224082459481 ...


### Part 1: How many of the available ingredient IDs are fresh?

Keep it simple: For each ingredient ID, check to see if it is contained in any of the ranges (then we'll see if this is fast enough):

In [33]:
def count_fresh_ingredients(fresh_ranges, available_ingredient_ids) -> int:
    """How many of the available ingredient IDs are in one of the fresh ranges?"""
    return quantify(any(lo <= id <= hi for (lo, hi) in fresh_ranges)
                    for id in available_ingredient_ids)

In [34]:
answer(5.1, 635, lambda:
       count_fresh_ingredients(fresh_ranges, available_ingredient_ids))

Puzzle  5.1:   .0076 seconds, answer 635             ok

Fast enough! Let's move on.

### Part 2: How many ingredient IDs are fresh?

In Part 2 we are asked how many of the possible ingredient IDs are fresh, regardless of whether the ID is available or not. I could convert a range like "3-5" into the set {3, 4, 5} and then union the sets together. How many elements would be in that set? Here's an estimate: 

In [35]:
sum(hi - lo for (lo, hi) in fresh_ranges)

476036797138761

OK, switch to Plan B. Instead of explicitly building this set I can instead sort the fresh ID ranges (by their low number) and go through them, keeping track of the lowest ID that has not yet been explored for freshness, and a count of the fresh IDs found so far. For each range, the number of new fresh IDs is the length of the range that starts with either the start of the range or the first unexplored ID, and ends at the end of the range.

In [36]:
def count_fresh_ids(fresh_ranges) -> int:
    """How many IDs are contained in the union of the ranges?"""
    fresh_count = 0 # The number of fresh IDs found so far
    unexplored  = 0 # The highest ID number that we haven't considered yet
    for (lo, hi) in sorted(fresh_ranges):
        start = max(unexplored, lo)
        fresh_count += len(range(start, hi + 1))
        unexplored = max(unexplored, hi + 1)
    return fresh_count

In [37]:
answer(5.2, 369761800782619, lambda:
       count_fresh_ids(fresh_ranges))

Puzzle  5.2:   .0001 seconds, answer 369761800782619 ok

# [Day 6](https://adventofcode.com/2025/day/5): Trash Compactor

Trash Compactor? [I've got a bad feeling  about this!](https://youtu.be/CZgeYSqUeTA?si=5UPS_HiCOmTKrEWX&t=32) We've fallen into a garbage smasher and have been asked to help some of the resident cephalopods children with their math homework. We can parse the homework worksheet, but we were told that the exact alignment of columns matters, so I'll keep each line as a string rather than converting it to a list of ints.

<img src="https://files.mastodon.social/media_attachments/files/115/674/362/938/836/432/original/3b83547aa749a914.jpg" width=500 alt="Gary Grady cartoon">

In [38]:
worksheet = parse(6, str)

────────────────────────────────────────────────────────────────────────────────────────────────────
Puzzle input ➜ 5 strs:
────────────────────────────────────────────────────────────────────────────────────────────────────
   4  82 68 85 74 6  56 14 2  8669 66 13 927  3 235 44   52 16  37  61 82 1774  96 15 775 789 91 ...
   6  87 39 72 56 12 69 79 58 4378 86 49 146  5 412 85 7751 577 69 813 55 9942 753 49 734 587 15 ...
 827 446 82 72 76 21 31 32 96 3257 21 21 171  2 31  17 9178 977 11 469 58  712 162 4    1 132 91 ...
9472 154 36 76  5 89 37  5 28    6 95 49  82 66 7   44 8183 524 38 819 11    7 825 9    8 397 48 ...
+    +   +  *  *  *  +  *  *  +    *  +  *   *  +   +  +    +   *  +   *  +    *   +  *   *   *  ...


### Part 1: What is the grand total of the answers to the individual problems?

We humans are used to arithmetic problems written in left-to-right order, but cephalopods use top-to-bottom order, with a postfix operator in the last row. We need to compute the sum or product for each column and add all those results together. So the problem in the first column above would be "`4 + 6 + 827 + 9472`". Here's the code:

In [39]:
def grand_total(worksheet):
    """The sum of the individual sum/product problems, where each column is a problem."""
    columns = T(map(str.split, worksheet)) # Columns are the transpose of the rows; `T` is transpose
    operations = {'*': prod, '+': sum}
    return sum(operations[op](mapt(int, numbers)) 
               for (*numbers, op) in columns)

In [40]:
answer(6.1, 5877594983578, lambda:
       grand_total(worksheet))

Puzzle  6.1:   .0014 seconds, answer 5877594983578   ok

### Part 2: What is the grand total of the answers to the individual problems with the new rules?

We learn that we did all the problems wrong. Cephalopodish number notation is different; numbers are read vertically rather than horizontally and the exact column alignment of each digit matters. Given the worksheet:

       4  82 68 85 74 6  56 14 2  8669 66 13 927  3 235 44 
       6  87 39 72 56 12 69 79 58 4378 86 49 146  5 412 85 
     827 446 82 72 76 21 31 32 96 3257 21 21 171  2 31  17 
    9472 154 36 76  5 89 37  5 28    6 95 49  82 66 7   44 
    +    +   +  *  *  *  +  *  *  +    *  +  *   *  +   +  

The problem in the leftmost column is not "`4 + 6 + 827 + 9472`"; rather it is "`9 + 84 + 27 + 4672`".

That means I can't just split each line into numbers, I'll have to be careful to maintain the blank spaces to the right and left of the digits, and I have to know in what position each column starts and ends. That part was tricky, so here's an explanation:
- I note from the worksheet above that each column starts at a position above  a `+` or `*` sign.
- Each column ends one character befpre the next `+` or `*` sign (one character is a blank space to separate columns).
- For the last column there is no terminator, so I'll add the string `' *'` to the operator line before computing the start positions.

In `grand_total2` I first break each line into columns, take the transpose of that (to give a list of problems), do the math on each problem, and return the sum of the results. Within `cephalopodish_math` I call `vertically`, whioch again does a transpose and then puts the digits together into an integer.

In [41]:
def grand_total2(worksheet: List[str]):
    """Solve the problem in each column with cephalopodish math and return the sum."""
    problems = T(break_into_columns(worksheet))
    return sum(map(cephalopodish_math, problems))

def break_into_columns(worksheet: List[str]) -> List[List[str]]:
    """Break a whole worksheet into columns by finding the `blanks` (the columns with
    only blank spaces) and then separating each line at these posisitons."""
    *number_lines, operator_line = worksheet
    operator_line = operator_line.ljust(len(worksheet[0])) + ' *'
    column_starts = [i for i, ch in enumerate(operator_line) if ch != ' ']
    return [break_line_into_columns(line, column_starts) for line in worksheet]

def break_line_into_columns(line: str, column_starts: List[int]) -> List[str]:
    """Break one line into columns, as specified by the list of `blanks` positions."""
    return [line[column_starts[i]:column_starts[i + 1]-1] 
            for i in range(len(column_starts) - 1)]

def cephalopodish_math(problem) -> int:
    """Return the sum or product of the vertically-arranged numbers."""
    *numbers, op = problem
    sum_or_prod = (sum if op.strip() == '+' else prod)
    return sum_or_prod(vertically(numbers))

def vertically(numbers: List[str]) -> List[int]:
    """Return a list of integers found by read numbers vertically by column."""
    return [int(cat(digits)) for digits in T(numbers)]

In [42]:
answer(6.2, 11159825706149, lambda:
       grand_total2(worksheet))

Puzzle  6.2:   .0039 seconds, answer 11159825706149  ok

I initially had an `IndexError` **bug** because the operator line is shorter then the numbers lines. Then I had an off-by-one **bug** that messed up the problem in the last column. To debug my errors I worked on the smaller example worksheet, doing things like this:

In [43]:
example = """\
123 328  51 64 
 45 64  387 23 
  6 98  215 314
*   +   *   +""".splitlines()

grand_total2(example)

3263827

In [44]:
break_into_columns(example)

[['123', '328', ' 51', '64 '],
 [' 45', '64 ', '387', '23 '],
 ['  6', '98 ', '215', '314'],
 ['*  ', '+  ', '*  ', '+']]

In [45]:
T(break_into_columns(example))

[('123', ' 45', '  6', '*  '),
 ('328', '64 ', '98 ', '+  '),
 (' 51', '387', '215', '*  '),
 ('64 ', '23 ', '314', '+')]

In [46]:
vertically(('123', 
            ' 45', 
            '  6'))

[1, 24, 356]

In [47]:
assert cephalopodish_math(('123', ' 45', '  6', '*  ')) == 1 * 24 * 356 == 8544

# [Day 7](https://adventofcode.com/2025/day/7): Laboratories

In the teleporter lab, we locate a diagram of a tachyon manifold, which is in need of repair. We obtain a diagram of the manifold, in which the `S` marks the start of the tachyon beams and a `^` is a splitter. 

I could parse this as a `Grid`, but I think I will just keep it as a list of strings. The main idea of a `Grid` is dealing with the 4 or 8 neighbors; a concept that this problem does not use.

In [48]:
manifold = parse(7)

────────────────────────────────────────────────────────────────────────────────────────────────────
Puzzle input ➜ 142 strs:
────────────────────────────────────────────────────────────────────────────────────────────────────
......................................................................S......................... ...
................................................................................................ ...
......................................................................^......................... ...
................................................................................................ ...
.....................................................................^.^........................ ...
................................................................................................ ...
....................................................................^.^.^....................... ...
..................................................................

### Part 1: How many times will the beam be split?

Tachyon beams move downwards unless they enounter a splitter, in which case they split into two beams, one immediately to the left of the splitter and one to the right, which each continue down from there. We're asked how many splits occur. If two beams end up in the same place they count as one beam, so if that beam is split again, that's just one more split, not two more.

That suggests I should keep a `set` of current beam positions, and update the set every time we go down one line. (It is a set rather than a list so that duplicate beam positins count as one, not as two.) I also need to keep track of a count of the number of splits, and return that at the end:

In [49]:
### Part 1: How many times will the beam be split?

def count_splits(manifold: List[str]) -> int:
    """How many beam split events occur as the beams make their way through the manifold?"""
    beams = {manifold[0].index('S')}
    split_count = 0
    for line in manifold:
        for b in list(beams):
            if line[b] == '^': # Replace the beam position with one on each side
                split_count += 1
                beams.remove(b)
                beams.update({b - 1, b + 1})
    return split_count

In [50]:
answer(7.1, 1681, lambda:
       count_splits(manifold))

Puzzle  7.1:   .0008 seconds, answer 1681            ok

### Part 2: In total, how many different timelines would a single tachyon particle end up on?

Now we're told this is a *quantum* tachyon manifold and we need to know how many different timelines a single tachyon appears in, or equivalently, how many different paths can the tachyon beams take. 

We can't just count the number of beams in the last line, because if a beam in position *b* takes two diffferent paths to get there, that counts as two, not one. Instead, I'll replace the `set` from Part 1 with a `Counter` of `{beam_position: number_of_paths_to_get_here}`. If a beam at position *b* with *n* paths to get there encounters a splitter, then in the next step both positions *b* - 1 and *b* + 1 are incremented by *n*. In the end we return the sum of all the numbers of paths.



In [51]:
def count_timelines(manifold: List[str]) -> int:
    """How many possible paths are there to the final line of the manifold?"""
    beams = Counter({manifold[0].index('S'): 1})
    for line in manifold:
        for b, n in list(beams.items()):
            if line[b] == '^':
                beams[b] = 0
                beams[b - 1] += n
                beams[b + 1] += n
    return sum(beams.values())

answer(7.2, 422102272495018, lambda:
       count_timelines(manifold))

Puzzle  7.2:   .0013 seconds, answer 422102272495018 ok

# Summary

In [52]:
summary(answers)

Puzzle  1.1:   .0005 seconds, answer 1182            ok
Puzzle  1.2:   .0009 seconds, answer 6907            ok
Puzzle  2.1:   .0030 seconds, answer 23560874270     ok
Puzzle  2.2:   .0041 seconds, answer 44143124633     ok
Puzzle  3.1:   .0006 seconds, answer 17085           ok
Puzzle  3.2:   .0020 seconds, answer 169408143086082 ok
Puzzle  4.1:   .0541 seconds, answer 1569            ok
Puzzle  4.2:   .1421 seconds, answer 9280            ok
Puzzle  5.1:   .0076 seconds, answer 635             ok
Puzzle  5.2:   .0001 seconds, answer 369761800782619 ok
Puzzle  6.1:   .0014 seconds, answer 5877594983578   ok
Puzzle  6.2:   .0039 seconds, answer 11159825706149  ok
Puzzle  7.1:   .0008 seconds, answer 1681            ok
Puzzle  7.2:   .0013 seconds, answer 422102272495018 ok

Time in seconds: total = 0.222, mean = 0.016, max = 0.142
